# Remote speech processing with local display and playback

This notebook should run properly in a Jupyter session running locally, or on a remote host (e.g. via binder.org). Speech processing is provided by the Parselmouth Praat library, and audio playback is on the client (javascript) side.

In [ ]:
import os
import urllib
from phonteract.plot import WaveformPlot
import parselmouth
import numpy as np
import pandas as pd
from bokeh.plotting import figure
from bokeh.layouts import column
from bokeh.palettes import Greys
from bokeh.models import BoxZoomTool, ColumnDataSource, LinearColorMapper, LogColorMapper
from bokeh.io import show, output_notebook

output_notebook()

In [ ]:
infile = 'resource/the_north_wind_and_the_sun.wav'
snd = parselmouth.Sound(infile)

Function required for embedded Bokeh app running on remote server. First, check the url bar and if the notebook is not running on localhost, expand the next cell and set `default_url` to the hostname, then execute. If running on localhost you can skip over the next cell.

In [ ]:
default_url = 'https://hub.gke.mybinder.org/'

def remote_jupyter_proxy_url(port):
    """
    Callable to configure Bokeh's show method when a proxy must be
    configured.

    If port is None we're asking about the URL
    for the origin header.
    """
    try:
        base_url = os.environ['EXTERNAL_URL']
    except KeyError:
        base_url = default_url
    host = urllib.parse.urlparse(base_url).netloc

    # If port is None we're asking for the URL origin
    # so return the public hostname.
    if port is None:
        return host

    service_url_path = os.environ['JUPYTERHUB_SERVICE_PREFIX']
    proxy_url_path = 'proxy/%d' % port

    user_url = urllib.parse.urljoin(base_url, service_url_path)
    full_url = urllib.parse.urljoin(user_url, proxy_url_path)
    return full_url

Proof of concept app that allows exploration of an audio waveform by zooming to a selected region. Press 'Play window' to play the current displayed waveform. Playback occurs in the javascript client, so should work even if backend is running on a remote server. Use the reset tool to restore to original zoom level.

In [ ]:
def wave_app(doc):
    wp = WaveformPlot(np.squeeze(snd.values), snd.sampling_frequency)
    wp.plot.plot_height = 200
    doc.add_root(
        column(wp.playbutton, wp.plot, wp.winstart_ti, wp.winend_ti, wp.fs_ti)
    )
    return doc

#show(wave_app)  # If running on localhost
show(wave_app, notebook_url=remote_jupyter_proxy_url)  # If running on a remote server

Another proof of concept. Show Praat processing on remote server for local display of a spectrogram.

In [ ]:
sgram = snd.to_spectrogram()
Sxx = sgram.values
freqs = sgram.y_bins().mean(axis=1)
times = sgram.x_bins().mean(axis=1)

In [ ]:
df = pd.DataFrame(index=freqs, data=Sxx, columns=times)
df = pd.DataFrame(df.stack(), columns=['amp']).reset_index()

palette = list(reversed(Greys[256]))
mapper = LogColorMapper(palette=palette, low=df.amp.min(), high=df.amp.max())

p = figure(title="Spectrogram",
    plot_width=600, plot_height=400,
    tools=[BoxZoomTool(dimensions='width'),'reset'],
    toolbar_location='below'
)
p.square(x="level_1", y="level_0", size=5,
    source=df,
    fill_color={'field': 'amp', 'transform': mapper},
    line_color=None
)

show(p)